# Discrete Fourier Transform with $\ell_1$-Norm

> **DFT$_{\ell_1}$** model.

**Published**: November 3, 2022

**Download**: This Jupyter notebook is at our GitHub repository. If you want to evaluate the code, please download the notebook from the [**tracebase**](https://github.com/xinychen/tracebase/blob/master/models/DFT-L1.ipynb) repository.

This notebook shows how to implement the Bayesian Temporal Matrix Factorization (BTMF), a fully Bayesian matrix factorization model, on some real-world data sets. To overcome the missing data problem in multivariate time series, BTMF takes into account both low-rank matrix structure and time series autoregression. For an in-depth discussion of BTMF, please see [1].

<div class="alert alert-block alert-info">
<font color="black">
<b>[1]</b> Guangcan Liu, Wayne Zhang (2022). <b>Recovery of future data via convolution nuclear norm minimization</b>. IEEE Transactions on Information Theory. <a href="https://doi.org/10.1109/TIT.2022.3196707" title="PDF"><b>[DOI]</b></a> 
</font>
</div>

In [ ]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def prox(z, lmbda):
    n = z.shape[0]
    z_hat = np.fft.fft(z)
    x_hat = np.zeros(n, dtype = complex)
    temp1 = np.abs(z_hat) - n / lmbda
    pos = np.where(temp1 > 0)
    temp2 = np.zeros(n)
    temp2[pos] = temp1[pos]
    x_hat = z_hat / np.abs(z_hat) * temp2
    return np.fft.ifft(x_hat).real

def mft(dense_mat, sparse_mat, lmbda, maxiter = 50):
    dim1, dim2 = sparse_mat.shape
    pos_train = np.where(sparse_mat != 0)
    if np.isnan(sparse_mat).any() == False:
        pos_test = np.where((dense_mat != 0) & (sparse_mat == 0))
    elif np.isnan(sparse_mat).any() == True:
        pos_test = np.where((dense_mat != 0) & (np.isnan(sparse_mat)))
        sparse_mat[np.isnan(sparse_mat)] = 0
    dense_test = dense_mat[pos_test]
    del dense_mat
    z = sparse_mat.copy()
    s = sparse_mat.copy()
    show_iter = 100
    for it in range(maxiter):
        x = prox((z - s / lmbda).reshape(-1, order = 'C'), 
                 lmbda).reshape([dim1, dim2], order = 'C')
        z = x + s / lmbda
        z[pos_train] = sparse_mat[pos_train]
        s = s + lmbda * (x - z)
        if (it + 1) % show_iter == 0:
            temp_hat = x[pos_test]
            print('Iter: {}'.format(it + 1))
            print('MAPE: {:.6}'.format(compute_mape(dense_test, temp_hat)))
            print('RMSE: {:.6}'.format(compute_rmse(dense_test, temp_hat)))
            print()
    return x

from ipywidgets import IntProgress
from IPython.display import display

def rolling4cast(dense_mat, sparse_mat, pred_step, delta, lmbda, maxiter):
    dim1, T = sparse_mat.shape
    start_time = T - pred_step
    max_count = int(np.ceil(pred_step / delta))
    mat_hat = np.zeros((dim1, max_count * delta))
    f = IntProgress(min = 0, max = max_count) # instantiate the bar
    display(f) # display the bar
    for t in range(max_count):
        temp1 = np.append(dense_mat[:, : start_time + t * delta], 
                          np.zeros((dim1, delta)), axis = 1)
        temp2 = np.append(sparse_mat[:, : start_time + t * delta], 
                          np.zeros((dim1, delta)), axis = 1)
        x = mft(temp1, temp2, lmbda, maxiter)
        mat_hat[:, t * delta : (t + 1) * delta] = x[:, - delta :]
        f.value = t
    small_dense_mat = dense_mat[:, start_time : T]
    pos = np.where((small_dense_mat != 0) & (np.invert(np.isnan(small_dense_mat))))
    mape = compute_mape(small_dense_mat[pos], mat_hat[pos])
    rmse = compute_rmse(small_dense_mat[pos], mat_hat[pos])
    print('Prediction MAPE: {:.6}'.format(mape))
    print('Prediction RMSE: {:.6}'.format(rmse))
    print()
    return mat_hat

In [ ]:
import numpy as np

dense_mat = np.load('../datasets/NYC-movement-data-set/hourly_speed_mat_2019_1.npz')['arr_0']
for month in range(2, 4):
    dense_mat = np.append(dense_mat, np.load('../datasets/NYC-movement-data-set/hourly_speed_mat_2019_{}.npz'.format(month))['arr_0'], axis = 1)
n = np.prod(dense_mat.shape)

import time
for delta in [6]:
    for lmbda in [0.5, 1, 5]:
        start = time.time()
        pred_step = 7 * 24
        maxiter = 50
        mat_hat = rolling4cast(dense_mat[:, : 24 * 7 * 10], dense_mat[:, : 24 * 7 * 10], 
                               pred_step, delta, lmbda * n, maxiter)
        print('lambda = {}'.format(lmbda))
        end = time.time()
        print('Running time: %d seconds'%(end - start))


In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize = (8, 2))
plt.plot(mat_hat[0, :])
plt.show()